<a href="https://colab.research.google.com/github/Mohammad-Kadiri/Handwritten-Digit-Recognition-using-Deep-Learning-with-TensorFlow/blob/main/Truck_detection_Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install YOLOv8 and required dependencies
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [4]:
import os
import torch
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow

In [5]:
# Load a pre-trained YOLOv8 model
model = YOLO('yolov8s.pt')

In [15]:
# Define the YAML content
yaml_content = """
train: /content/images/train
val: /content/images/val

nc: 2  # Number of classes (truck and not truck)
names: ['truck', 'not_truck']  # Class names
"""

# Write the YAML content to a file
with open('/content/images.yaml', 'w') as f:
    f.write(yaml_content)

In [16]:
def write_yolo_label(image_name, class_id, bbox, label_dir):
    """
    Args:
    - image_name: Name of the image file (e.g., 'image1.jpg')
    - class_id: Integer class ID (e.g., 0 for 'truck', 1 for 'not truck')
    - bbox: Tuple containing (x_center, y_center, width, height) all normalized between 0 and 1
    - label_dir: Directory where the label file should be saved
    """
    # Extract the image name without extension
    label_file = os.path.splitext(image_name)[0] + '.txt'

    # Create the full path for the label file
    label_path = os.path.join(label_dir, label_file)

    # Write the label information to the file
    with open(label_path, 'a') as f:  # 'a' mode to append if there are multiple objects
        # Write in YOLO format: class_id x_center y_center width height
        f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")


In [13]:
import os

# Set directories
image_dir = '/content/images/train'
label_dir = '/content/labels/train'

# Create label directory if it doesn't exist
os.makedirs(label_dir, exist_ok=True)

# Define class names and IDs
class_names = ['truck', 'not_truck']
class_ids = {name: i for i, name in enumerate(class_names)}

# Function to create label files based on the order of images
def create_labels(image_dir, label_dir, switch_index):
    # List all image files
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))])

    # Create labels
    for i, image_file in enumerate(image_files):
        # Determine class based on the index
        if i < switch_index:
            label_text = 'truck'
        else:
            label_text = 'not_truck'

        # Create a label file with the corresponding class ID
        label_file = os.path.join(label_dir, os.path.splitext(image_file)[0] + '.txt')
        with open(label_file, 'w') as f:
            # Write label in YOLO format (assuming a single object per image)
            class_id = class_ids[label_text]
            f.write(f"{class_id} 0.5 0.5 1 1\n")  # Example: center of the image, full width/height

# Ask the user for the switch index
switch_index = int(input("Enter the index where 'truck' ends and 'not_truck' begins: "))

# Call the function to create label files
create_labels(image_dir, label_dir, switch_index)

print("Labels have been created successfully!")

Enter the index where 'truck' ends and 'not_truck' begins: 22
Labels have been created successfully!


In [14]:
import os

# Set directories
image_dir = '/content/images/val'
label_dir = '/content/labels/val'

# Create label directory if it doesn't exist
os.makedirs(label_dir, exist_ok=True)

# Define class names and IDs
class_names = ['truck', 'not_truck']
class_ids = {name: i for i, name in enumerate(class_names)}

# Function to create label files based on the order of images
def create_labels(image_dir, label_dir, switch_index):
    # List all image files
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))])

    # Create labels
    for i, image_file in enumerate(image_files):
        # Determine class based on the index
        if i < switch_index:
            label_text = 'truck'
        else:
            label_text = 'not_truck'

        # Create a label file with the corresponding class ID
        label_file = os.path.join(label_dir, os.path.splitext(image_file)[0] + '.txt')
        with open(label_file, 'w') as f:
            # Write label in YOLO format (assuming a single object per image)
            class_id = class_ids[label_text]
            f.write(f"{class_id} 0.5 0.5 1 1\n")  # Example: center of the image, full width/height

# Ask the user for the switch index
switch_index = int(input("Enter the index where 'truck' ends and 'not_truck' begins: "))

# Call the function to create label files
create_labels(image_dir, label_dir, switch_index)

print("Labels have been created successfully!")

Enter the index where 'truck' ends and 'not_truck' begins: 17
Labels have been created successfully!


In [18]:
# Fine-tune YOLOv8 on your custom dataset
results = model.train(data='/content/images.yaml', epochs=50, imgsz=640)

Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/images.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

train: Scanning /content/labels/train... 32 images, 12 backgrounds, 0 corrupt: 100%|██████████| 44/44 [00:00<00:00, 1712.88it/s]

train: New cache created: /content/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/labels/val... 22 images, 11 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<00:00, 855.49it/s]


val: New cache created: /content/labels/val.cache
Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      4.14G     0.2312      0.652      1.006          7        640: 100%|██████████| 3/3 [00:00<00:00,  4.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.00it/s]

                   all         33         22      0.881        0.5      0.397      0.345



50 epochs completed in 0.038 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 22.5MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.46it/s]


                   all         33         22      0.363        0.6      0.422      0.357
                 truck         17         17      0.599          1      0.798      0.689
             not_truck          5          5      0.127        0.2     0.0468     0.0254
Speed: 0.2ms preprocess, 5.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train3


In [22]:
from PIL import Image

# Load an image
img_path = '/content/test/img032.png'
img = Image.open(img_path)

In [23]:
results = model(img)


0: 640x640 1 not_truck, 20.0ms
Speed: 3.9ms preprocess, 20.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
